<a href="https://colab.research.google.com/github/Santimalli-Gowthami/RPS-by-Gowthami/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip uninstall -y tf-nightly tensorflow
  !pip install tensorflow
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load and prepare data
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

train_messages = train_data['message'].values
train_labels = train_data['label'].values
test_messages = test_data['message'].values
test_labels = test_data['label'].values

# Text Vectorization
max_features = 10000
sequence_length = 100

vectorizer = keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

vectorizer.adapt(train_messages)
train_sequences = vectorizer(train_messages)
test_sequences = vectorizer(test_messages)

# Build model
model = keras.Sequential([
    keras.layers.Embedding(max_features, 128, input_length=sequence_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_sequences,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(test_sequences, test_labels),
    callbacks=[early_stop],
    verbose=1
)

# Evaluate
loss, accuracy = model.evaluate(test_sequences, test_labels, verbose=0)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

# Define predict_message function
def predict_message(pred_text):
    pred_sequence = vectorizer([pred_text])
    prediction = model.predict(pred_sequence, verbose=0)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    return [float(prediction), label]


In [ ]:
# =============================================================================
# CELL 3 - REPLACE YOUR CURRENT CELL 3 WITH THIS EXACT CODE
# =============================================================================

# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_sequence = vectorizer([pred_text])
    prediction = model.predict(pred_sequence, verbose=0)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    return [float(prediction), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
